In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

このコードは計算グラフを作るだけで、実際変数でさえ初期化されていない。このグラフを評価するには、`TensorFlow セッション`を開き、それを使用して変数を初期化し、fを評価する。

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
result

42

In [6]:
sess.close()

In [9]:
# 別のやり方1
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [11]:
# 別のやり方2(Jupyterはこれを推奨)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
result

42